In [26]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

In [27]:
sheet_id = "1ujxQncgOQLoOXZpWNR46fNeJxpDwnfCamlTT5Q0uO3E"
sheet_name = "yf_ticker"

In [28]:
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [29]:
df = pd.read_csv(url)
df

,set50,set100,mai
0,ADVANC.BK,AAV.BK,AU.BK
1,AOT.BK,ACE.BK,JCKH.BK
2,AWC.BK,ADVANC.BK,KASET.BK
3,BANPU.BK,AMATA.BK,MUD.BK
4,BBL.BK,AOT.BK,NTSC.BK
...,...,...,...
201,NaN,NaN,SICT.BK
202,NaN,NaN,SIMAT.BK
203,NaN,NaN,SPVI.BK
204,NaN,NaN,TPS.BK


In [30]:
df_set50 = df["set50"].dropna()
df_set50

0     ADVANC.BK
1        AOT.BK
2        AWC.BK
3      BANPU.BK
4        BBL.BK
5       BDMS.BK
6        BEM.BK
7      BGRIM.BK
8         BH.BK
9        BTS.BK
10       CBG.BK
11    CENTEL.BK
12      COM7.BK
13     CPALL.BK
14       CPF.BK
15       CPN.BK
16       CRC.BK
17     DELTA.BK
18        EA.BK
19      EGCO.BK
20    GLOBAL.BK
21      GPSC.BK
22      GULF.BK
23     HMPRO.BK
24    INTUCH.BK
25       IVL.BK
26     JMART.BK
27       JMT.BK
28     KBANK.BK
29       KTB.BK
30       KTC.BK
31        LH.BK
32      MINT.BK
33       MTC.BK
34        OR.BK
35       OSP.BK
36       PTT.BK
37     PTTEP.BK
38     PTTGC.BK
39     RATCH.BK
40     SAWAD.BK
41       SCB.BK
42       SCC.BK
43      SCGP.BK
44    TIDLOR.BK
45     TISCO.BK
46       TOP.BK
47      TRUE.BK
48       TTB.BK
49        TU.BK
Name: set50, dtype: object

In [31]:
set50_list = df_set50.values.tolist()
set50_list

['ADVANC.BK',
 'AOT.BK',
 'AWC.BK',
 'BANPU.BK',
 'BBL.BK',
 'BDMS.BK',
 'BEM.BK',
 'BGRIM.BK',
 'BH.BK',
 'BTS.BK',
 'CBG.BK',
 'CENTEL.BK',
 'COM7.BK',
 'CPALL.BK',
 'CPF.BK',
 'CPN.BK',
 'CRC.BK',
 'DELTA.BK',
 'EA.BK',
 'EGCO.BK',
 'GLOBAL.BK',
 'GPSC.BK',
 'GULF.BK',
 'HMPRO.BK',
 'INTUCH.BK',
 'IVL.BK',
 'JMART.BK',
 'JMT.BK',
 'KBANK.BK',
 'KTB.BK',
 'KTC.BK',
 'LH.BK',
 'MINT.BK',
 'MTC.BK',
 'OR.BK',
 'OSP.BK',
 'PTT.BK',
 'PTTEP.BK',
 'PTTGC.BK',
 'RATCH.BK',
 'SAWAD.BK',
 'SCB.BK',
 'SCC.BK',
 'SCGP.BK',
 'TIDLOR.BK',
 'TISCO.BK',
 'TOP.BK',
 'TRUE.BK',
 'TTB.BK',
 'TU.BK']

In [32]:
# Import All Stocks Universe
data = yf.download(set50_list, period='2y' , interval='1d',group_by='column')

# Cleaning DataFrame
data.fillna(method="ffill",inplace=True)
df = data.stack()
stock_df = df.sort_index(level=1)
stock_df.index.names = ['Date','Stock']


[*********************100%***********************]  50 of 50 completed


In [33]:
df = data.asfreq('D' , method = 'ffill')
name = df.columns.levels[1]
df.columns = df.columns.set_levels(  name.str.replace( '.BK' , '') ,  level=1  )
df_max  = df['High'].rolling(52*7).max()
df_min  = df['Low'].rolling(52*7).min()


In [35]:
drop_from_high =  df.iloc[-1]['Close'] / df_max.iloc[-1]

In [37]:
fig = px.scatter(  drop_from_high )
fig.show()

In [55]:
stock_df['52W-High'] = stock_df.groupby('Stock')['Close'].transform(lambda x: x.rolling(52*7, closed='left').max())
stock_df['52W-Low'] = stock_df.groupby('Stock')['Close'].transform(lambda x: x.rolling(52*7, closed='left').min())
stock_df['NH'] = stock_df['Close'] > stock_df['52W-High'].shift(1)
stock_df['NL'] = stock_df['Close'] < stock_df['52W-Low'].shift(1)

breadth_df1 = stock_df.groupby(by=['Date'])['NH'].sum()
breadth_df2 = stock_df.groupby(by=['Date'])['NL'].sum()
breadth_df3= pd.concat([breadth_df1,breadth_df2],join='inner',axis=1)
breadth_df3 = breadth_df3.iloc[52*7:]
breadth_df3['High-Low index'] = (breadth_df3['NH']/(breadth_df3['NH']+breadth_df3['NL']))*100
breadth_df3 = breadth_df3.reset_index().fillna(method = 'ffill')

In [57]:
fig = px.line(breadth_df3, y='High-Low index', x='Date',template= "simple_white",title='MARKET BREADTH : High-Low index')

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"])] )

fig.update_xaxes(rangeslider_visible=True)
fig.show()